In [107]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from pandas.plotting import scatter_matrix

In [108]:
stock = pd.read_csv('data/nasdaq/AAPL.csv', index_col=0)
stock = stock[-1500:]
stock.index = pd.to_datetime(stock.index, format="%d-%m-%Y")
stock['MA50'] = stock['Open'].rolling(50).mean()
stock['MA200'] = stock['Open'].rolling(200).mean()
stock.head()

,Low,Open,Volume,High,Close,Adjusted Close,MA50,MA200
Date,,,,,,,,
2015-11-11,28.802500,29.092501,180872000,29.355000,29.027500,26.767387,NaN,NaN
2015-11-12,28.912500,29.065001,130102400,29.205000,28.930000,26.677477,NaN,NaN
2015-11-13,28.067499,28.799999,183249600,28.892500,28.084999,25.898272,NaN,NaN
2015-11-16,27.750000,27.844999,152426800,28.559999,28.545000,26.322456,NaN,NaN
2015-11-17,28.330000,28.730000,110467600,28.762501,28.422501,26.209492,NaN,NaN


# Definition of the portfolio

In [109]:
'''
Class of a Portfolio, 
Contains its value and what stocks there is inside.
value is a float
content is a dictionary with the symbol as a key and value the number of action in the portfolio
'''
class Portfolio:
    def __init__(self, value, content):
        self.__value = value
        self.__content = content
        
    def get_value(self):
        return self.__value
        
    def buy(self, action, number, price):
        print("BUY: " + str(number) + "\/ " + str(price))
        self.__value -= price*number
        self.__content[action] += number
        
    def sell(self, action, number, price):
        print("SELL: " + str(number) + "\/ " + str(price))
        self.__value += price*number
        self.__content[action] -= number
        
    def close_position(self, price):
        for i in self.__content.keys():
            if self.__content[i] > 0:
                self.sell(i, self.__content[i], price)
            elif self.__content[i] < 0:
                self.buy(i, -self.__content[i], price)
                

        

In [110]:
def strategy(stock, old_info, portfolio, date):
    if stock["MA50"][date]-stock["MA200"][date] > 0:
        new_info = True
    else:
        new_info = False
    
    if old_info^new_info:
        if not(new_info):
            portfolio.sell("AAPL", 10, stock["Open"][date])
        else: 
            portfolio.buy("AAPL", 10, stock["Open"][date])
    return new_info, portfolio
            

In [111]:
def simulator(strategy, stocks):
    portfolio = Portfolio(0, {"AAPL":0})
    old_info = False
    for date in stocks.index[-1300:]:
        old_info, portfolio = strategy(stocks, old_info, portfolio, date)
        
    portfolio.close_position(stock["Open"][date])
    print(portfolio.get_value())

In [112]:
simulator(strategy, stock)

BUY: 10\/ 26.975000381469727
SELL: 10\/ 39.21500015258789
BUY: 10\/ 50.47499847412109
SELL: 10\/ 149.3300018310547
1110.9500312805176
